In [111]:
global file
file = input('Selet a folder') ##enter your file path here - the file is in the repo "summary_stats.csv".
raw_file = file
##file

In [85]:
from datetime import datetime
from datetime import date
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import math
import pandas as pd
import  os as os

In [86]:
sg = Subgrounds()
snapshot = sg.load_api('https://hub.snapshot.org/graphql')

In [87]:
snapshot.Proposal.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.STRING,
  snapshot.Proposal.end,
)

In [88]:
spacename = input('spacename plz?')
proposals = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=10000,
  where=[
    snapshot.Proposal.space == spacename, ##'fuse.eth',
    snapshot.Proposal.state == 'closed'
    ##snapshot.Proposal.title == 'OIP-18: Reward rate framework and reduction',
  ]
)

In [89]:
proposals_snapshots = sg.query_df([
    proposals.title,
    proposals.id,
    proposals.body,
    proposals.scores,
    proposals.scores_total
])

In [90]:
proposals_choices = sg.query(proposals.choices)

In [91]:
proposals_choices = pd.DataFrame(proposals_choices)
##proposals_choices = pd.DataFrame(proposals_choices, columns = ['option_1', 'option_2', 'option_3', 'option_4', 'option_5','option_6','option_7','option_8','option_9','option_10','option_11','option_12','option_13','option_14','option_15','option_16','option_17','option_18','option_19','option_20','option_21','option_22'])

In [92]:
olympus_governance_view = pd.concat([proposals_snapshots,proposals_choices], axis=1)

In [93]:
##let's view the output just to make sure
olympus_governance_view

,proposals_title,proposals_id,proposals_body,proposals_scores,proposals_scores_total,0,1,2,3,4,5,6
0,OIP-94A: Amend Interim Ranged Stability Policy...,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...,**Summary:**\n\nGrant Policy Authority to:\nCo...,7.511731,359890.743297,Reject Amendment,Approve Amendment,None,None,None,None,None
1,OIP-94A: Amend Interim Ranged Stability Policy...,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...,**Summary:**\n\nGrant Policy Authority to:\nCo...,359883.231566,359890.743297,For: Deploy Bond Protocol,Against: Do nothing,None,None,None,None,None
2,OIP-104: Deploy Permissionless OP as Bond Prot...,0xcce2dd72fa5bf12ddbe5a15ed23f6256f6cd841ed572...,#### Summary\nDeploy the new system of Permiss...,292512.593605,296088.544685,For: In support of proposal,Against: Not in support,None,None,None,None,None
3,OIP-104: Deploy Permissionless OP as Bond Prot...,0xcce2dd72fa5bf12ddbe5a15ed23f6256f6cd841ed572...,#### Summary\nDeploy the new system of Permiss...,3575.951080,296088.544685,Approve $50k Grant to Artemis,Reject,None,None,None,None,None
4,OIP-103 Balancer Migration,0xebd05bd5896c472eecceb380b345689fc5afcd295865...,**Summary**\n\nDesignate Balancer as the platf...,322598.605158,322598.605158,For,Against,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
460,Add DAI Bonds,QmUC5xa4YaWZBV7fy241svBQ8bHSMksm8kB7avk1WyWv9m,Summary: We can launch bonds using DAI instead...,7.353389,26630.139267,NaN,NaN,NaN,NaN,NaN,NaN,NaN
461,Add DAI Bonds,QmUC5xa4YaWZBV7fy241svBQ8bHSMksm8kB7avk1WyWv9m,Summary: We can launch bonds using DAI instead...,0.000000,26630.139267,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462,Open an OHM-ETH Pool on Uniswap,QmVaYYojHUCqVhM835eSWtih6CyF1koQpSinPg9Kyw21RN,Summary:\nSeveral community members have broug...,177.543607,20614.265541,NaN,NaN,NaN,NaN,NaN,NaN,NaN
463,Open an OHM-ETH Pool on Uniswap,QmVaYYojHUCqVhM835eSWtih6CyF1koQpSinPg9Kyw21RN,Summary:\nSeveral community members have broug...,0.000000,20614.265541,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
path =file+'/'+spacename+'_proposals_table_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(olympus_governance_view))+'_proposals.csv'
olympus_governance_view.to_csv(path, index = False)

In [95]:
total_proposals = len(olympus_governance_view)
total_proposals

465

In [96]:
proposal_id = olympus_governance_view.iloc[0,1]

In [97]:
print(proposal_id)

0xe6233a0b159f58a97ba41019bd09cb904bd0120f656dcaac3c7aa33d862abf72


In [98]:

vote_tracker = snapshot.Query.votes(
orderBy = 'created',
orderDirection='desc',
first=10000,
where=[
  snapshot.Vote.proposal == proposal_id
]
)

In [99]:
voting_snapshots_list = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice,
    vote_tracker.vp
])

In [100]:
voting_snapshots_list

,votes_id,votes_voter,votes_created,votes_choice,votes_vp
0,bafkreih44a44m4ej2loni54bjgzz2nmbfc2lit324hiqh...,0xCc0002972d604DceD87f079306ad6a555C9Fe122,1658759354,2,376.437434
1,0x473f04dd52469588cc54a10017482b98ce083ff1eb0d...,0xd8929472A97db29867fE5C6F18922c24F59A7A71,1658757220,2,1057.429981
2,0xbe37fd97164f24b6ee13f4b633ad3ff9413493c5069c...,0xBDbFdf3e82fC9d2bE1352e252aB1Ce2287fC2122,1658755919,2,87.229162
3,0x121f54a30024ec4166d1ac4449a3d8293bf4a85d6753...,0x903A97385f13597ea34e1952C6B50D303498FceB,1658753952,2,435.343234
4,0xbbafaefe287c9ce99473382b0b11356d8e8dbb4f9de7...,0x627D5Ed9d7b2EC21a785aB4C560492B3eB65fB3f,1658753659,2,84.418999
...,...,...,...,...,...
112,bafkreieq3vfnro7pspk2m53eg2bacmmmy4ttw5w4hxr5v...,0x6Fbc794c8FE80ba45B6398c54402dc1E167F1572,1658428102,2,317.437320
113,0xfc5b5b03ea7784ad1245f4f1503768186592bed8160d...,0x0199bE12d6F574DB4A55b061653BeC89bFF62aF7,1658427939,2,323.369849
114,bafkreicqpuswkucns5vcspnixuwu4hx6dqhxu6ucpx3yp...,0x55B58545C37a23Ab68eB380116aaCE5eaF7FF564,1658427862,2,340.931001
115,0x812aa20fa9e6144646a93305df9806feebccef2c487b...,0xca6aA0576c2ADc79d2c20e6D53c07f590f62932A,1658427581,2,8671.132415


In [101]:
voting_snapshots_list['Proposal'] = proposal_id
voting_snapshots_list

,votes_id,votes_voter,votes_created,votes_choice,votes_vp,Proposal
0,bafkreih44a44m4ej2loni54bjgzz2nmbfc2lit324hiqh...,0xCc0002972d604DceD87f079306ad6a555C9Fe122,1658759354,2,376.437434,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...
1,0x473f04dd52469588cc54a10017482b98ce083ff1eb0d...,0xd8929472A97db29867fE5C6F18922c24F59A7A71,1658757220,2,1057.429981,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...
2,0xbe37fd97164f24b6ee13f4b633ad3ff9413493c5069c...,0xBDbFdf3e82fC9d2bE1352e252aB1Ce2287fC2122,1658755919,2,87.229162,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...
3,0x121f54a30024ec4166d1ac4449a3d8293bf4a85d6753...,0x903A97385f13597ea34e1952C6B50D303498FceB,1658753952,2,435.343234,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...
4,0xbbafaefe287c9ce99473382b0b11356d8e8dbb4f9de7...,0x627D5Ed9d7b2EC21a785aB4C560492B3eB65fB3f,1658753659,2,84.418999,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...
...,...,...,...,...,...,...
112,bafkreieq3vfnro7pspk2m53eg2bacmmmy4ttw5w4hxr5v...,0x6Fbc794c8FE80ba45B6398c54402dc1E167F1572,1658428102,2,317.437320,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...
113,0xfc5b5b03ea7784ad1245f4f1503768186592bed8160d...,0x0199bE12d6F574DB4A55b061653BeC89bFF62aF7,1658427939,2,323.369849,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...
114,bafkreicqpuswkucns5vcspnixuwu4hx6dqhxu6ucpx3yp...,0x55B58545C37a23Ab68eB380116aaCE5eaF7FF564,1658427862,2,340.931001,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...
115,0x812aa20fa9e6144646a93305df9806feebccef2c487b...,0xca6aA0576c2ADc79d2c20e6D53c07f590f62932A,1658427581,2,8671.132415,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...


In [102]:
x=0
while x <total_proposals:
##print(olympus_governance_view.iloc[2,1])
    proposal_id = olympus_governance_view.iloc[x,1]

    vote_tracker = snapshot.Query.votes(
    orderBy = 'created',
    orderDirection='desc',
    first=10000,
    where=[
      snapshot.Vote.proposal == proposal_id
    ]
    )
    voting_snapshots = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice,
    vote_tracker.vp
    ])

    voting_snapshots['Proposal'] = proposal_id
    voting_snapshots_list=pd.concat([voting_snapshots_list, voting_snapshots])
    ##voting_snapshots['Proposal'] = proposal_id

#    pd.concat([voting_snapshots_list, voting_snapshots])
    x=x+1
    progress = 100*(round(x/total_proposals,4))
    ##clear_output(wait=True)
    if progress%10==0
        print("Progress",progress,"%")

Progress 0.22 %
Progress 0.43 %
Progress 0.65 %
Progress 0.86 %
Progress 1.08 %
Progress 1.29 %
Progress 1.51 %
Progress 1.72 %
Progress 1.94 %
Progress 2.15 %
Progress 2.37 %
Progress 2.58 %
Progress 2.8000000000000003 %
Progress 3.01 %
Progress 3.2300000000000004 %
Progress 3.44 %
Progress 3.66 %
Progress 3.8699999999999997 %
Progress 4.09 %
Progress 4.3 %
Progress 4.52 %
Progress 4.73 %
Progress 4.95 %
Progress 5.16 %
Progress 5.38 %
Progress 5.59 %
Progress 5.81 %
Progress 6.02 %
Progress 6.239999999999999 %
Progress 6.45 %
Progress 6.67 %
Progress 6.88 %
Progress 7.1 %
Progress 7.31 %
Progress 7.53 %
Progress 7.739999999999999 %
Progress 7.960000000000001 %
Progress 8.17 %
Progress 8.39 %
Progress 8.6 %
Progress 8.82 %
Progress 9.030000000000001 %
Progress 9.25 %
Progress 9.46 %
Progress 9.68 %
Progress 9.89 %
Progress 10.11 %
Progress 10.32 %
Progress 10.54 %
Progress 10.75 %
Progress 10.97 %
Progress 11.18 %
Progress 11.4 %
Progress 11.61 %
Progress 11.83 %
Progress 12.04 %
Prog

In [103]:
len(voting_snapshots_list)

102478

In [104]:
path =file+'/'+spacename+'_voting_snapshots_list_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(olympus_governance_view))+'.csv'
voting_snapshots_list.to_csv(path, index = False)

In [105]:
governance_data = pd.merge(voting_snapshots_list, olympus_governance_view, how='inner', left_on='Proposal', right_on='proposals_id')
governance_data

,votes_id,votes_voter,votes_created,votes_choice,votes_vp,Proposal,proposals_title,proposals_id,proposals_body,proposals_scores,proposals_scores_total,0,1,2,3,4,5,6
0,bafkreih44a44m4ej2loni54bjgzz2nmbfc2lit324hiqh...,0xCc0002972d604DceD87f079306ad6a555C9Fe122,1.658759e+09,2,376.437434,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...,OIP-94A: Amend Interim Ranged Stability Policy...,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...,**Summary:**\n\nGrant Policy Authority to:\nCo...,7.511731,359890.743297,Reject Amendment,Approve Amendment,None,None,None,None,None
1,bafkreih44a44m4ej2loni54bjgzz2nmbfc2lit324hiqh...,0xCc0002972d604DceD87f079306ad6a555C9Fe122,1.658759e+09,2,376.437434,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...,OIP-94A: Amend Interim Ranged Stability Policy...,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...,**Summary:**\n\nGrant Policy Authority to:\nCo...,359883.231566,359890.743297,For: Deploy Bond Protocol,Against: Do nothing,None,None,None,None,None
2,0x473f04dd52469588cc54a10017482b98ce083ff1eb0d...,0xd8929472A97db29867fE5C6F18922c24F59A7A71,1.658757e+09,2,1057.429981,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...,OIP-94A: Amend Interim Ranged Stability Policy...,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...,**Summary:**\n\nGrant Policy Authority to:\nCo...,7.511731,359890.743297,Reject Amendment,Approve Amendment,None,None,None,None,None
3,0x473f04dd52469588cc54a10017482b98ce083ff1eb0d...,0xd8929472A97db29867fE5C6F18922c24F59A7A71,1.658757e+09,2,1057.429981,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...,OIP-94A: Amend Interim Ranged Stability Policy...,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...,**Summary:**\n\nGrant Policy Authority to:\nCo...,359883.231566,359890.743297,For: Deploy Bond Protocol,Against: Do nothing,None,None,None,None,None
4,0xbe37fd97164f24b6ee13f4b633ad3ff9413493c5069c...,0xBDbFdf3e82fC9d2bE1352e252aB1Ce2287fC2122,1.658756e+09,2,87.229162,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...,OIP-94A: Amend Interim Ranged Stability Policy...,0xe6233a0b159f58a97ba41019bd09cb904bd0120f656d...,**Summary:**\n\nGrant Policy Authority to:\nCo...,7.511731,359890.743297,Reject Amendment,Approve Amendment,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242780,QmXPL7fViDQBjbg4EikDftXtTwht1sSUYVtHcgrzU3WrUy,0x3942Ae3782FbD658CC19A8Db602D937baF7CB57A,1.617555e+09,{'1': 1},94.118185,QmVaYYojHUCqVhM835eSWtih6CyF1koQpSinPg9Kyw21RN,Open an OHM-ETH Pool on Uniswap,QmVaYYojHUCqVhM835eSWtih6CyF1koQpSinPg9Kyw21RN,Summary:\nSeveral community members have broug...,0.000000,20614.265541,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242781,QmXPL7fViDQBjbg4EikDftXtTwht1sSUYVtHcgrzU3WrUy,0x3942Ae3782FbD658CC19A8Db602D937baF7CB57A,1.617555e+09,{'1': 1},94.118185,QmVaYYojHUCqVhM835eSWtih6CyF1koQpSinPg9Kyw21RN,Open an OHM-ETH Pool on Uniswap,QmVaYYojHUCqVhM835eSWtih6CyF1koQpSinPg9Kyw21RN,Summary:\nSeveral community members have broug...,165.462663,20614.265541,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242782,QmYwVd9zd2AzAYSBncxeWYnePQQNGTPJiKjywXdKoFpJvv,0x45ADFF324Eb1ac03a6A115dc539052232D4bA980,1.617554e+09,{'1': 1},0.842478,QmVaYYojHUCqVhM835eSWtih6CyF1koQpSinPg9Kyw21RN,Open an OHM-ETH Pool on Uniswap,QmVaYYojHUCqVhM835eSWtih6CyF1koQpSinPg9Kyw21RN,Summary:\nSeveral community members have broug...,177.543607,20614.265541,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242783,QmYwVd9zd2AzAYSBncxeWYnePQQNGTPJiKjywXdKoFpJvv,0x45ADFF324Eb1ac03a6A115dc539052232D4bA980,1.617554e+09,{'1': 1},0.842478,QmVaYYojHUCqVhM835eSWtih6CyF1koQpSinPg9Kyw21RN,Open an OHM-ETH Pool on Uniswap,QmVaYYojHUCqVhM835eSWtih6CyF1koQpSinPg9Kyw21RN,Summary:\nSeveral community members have broug...,0.000000,20614.265541,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
final_file = file+'\\'+'final'
final_raw_file = final_file

In [123]:
os.makedirs(final_raw_file, exist_ok=True)

In [124]:
final_path =final_file+'\\'+spacename+'governance_data_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(olympus_governance_view))+'.csv'

In [125]:
governance_data.to_csv(final_path, index = False)

In [132]:
import pandasql as ps
import duckdb as db

In [145]:
crunch_data = db.query("select "
                           "Proposal"
                           ",votes_voter"
                           ", votes_created"
                           ",sum(votes_created) over (Partition by Proposal order by votes_created) as cumulative_vp"
                           ", sum(votes_created) over (Partition by Proposal) as total_vp"
                           ", (votes_created::decimal/sum(votes_created::decimal) over (Partition by Proposal)) as percentange_vp  "
                       "from "
                       "    governance_data "
                       ""
                       "Group by "
                       "    Proposal,votes_voter, votes_created "
                       ""
                       "Order by "
                       "    Proposal, "
                       "    votes_created desc "
                       "").df()



crunch_data

,Proposal,votes_voter,votes_created,cumulative_vp,total_vp,percentange_vp
0,0x00fd601fd700678a9d6b7e484e742ccbbcf2f7f27c6d...,0x930914b865db5FE9A15a42A99481f77F469C9F04,1.646728e+09,1.416022e+11,1.416022e+11,0.011629
1,0x00fd601fd700678a9d6b7e484e742ccbbcf2f7f27c6d...,0x40C848ae24c433b6B1806B4DDDb7f5B38Afcffc1,1.646716e+09,1.399555e+11,1.416022e+11,0.011629
2,0x00fd601fd700678a9d6b7e484e742ccbbcf2f7f27c6d...,0xD14841F625e88394933b27f2C04d3C029A830697,1.646705e+09,1.383088e+11,1.416022e+11,0.011629
3,0x00fd601fd700678a9d6b7e484e742ccbbcf2f7f27c6d...,0x7672B17D5Eebb7194d57066f86FBcb4e85a7e10E,1.646704e+09,1.366621e+11,1.416022e+11,0.011629
4,0x00fd601fd700678a9d6b7e484e742ccbbcf2f7f27c6d...,0x5f4b7d6cc4eE935f48e4AE4CdEfc3b96DfCe0550,1.646702e+09,1.350154e+11,1.416022e+11,0.011629
...,...,...,...,...,...,...
45930,Qmen3Ps2nCXLGYwi1dcGkaVTjtdfTFdp4DdG9cJSQBSZX3,0xbAEd0eaC0D1B90CAcAd294f783fe545c37019212,1.623777e+09,8.118883e+09,4.222204e+11,0.003846
45931,Qmen3Ps2nCXLGYwi1dcGkaVTjtdfTFdp4DdG9cJSQBSZX3,0x79463163BEF8433D5F0808D6377D01E9B466a857,1.623777e+09,6.495106e+09,4.222204e+11,0.003846
45932,Qmen3Ps2nCXLGYwi1dcGkaVTjtdfTFdp4DdG9cJSQBSZX3,0xe8d0587D82Ae54b0dd1F8E234bA3f0Ce1E2f047A,1.623777e+09,4.871330e+09,4.222204e+11,0.003846
45933,Qmen3Ps2nCXLGYwi1dcGkaVTjtdfTFdp4DdG9cJSQBSZX3,0x00903F96abe2743b81fc7Fcf4874c715A3fe1C88,1.623777e+09,3.247553e+09,4.222204e+11,0.003846
